In [1]:
import tkinter as tk
import copy as cp
from heapq import heappush, heappop
import random
from tkinter import ttk
from collections import defaultdict


class Graph(object):
    MAX_EDGE_WEIGHT = 100
    MAX_XY_SCREEN = 500

    def __init__(self, nodes : int):        
        # The default dictionary would create an empty list as a default (value) for the nonexistent keys.
        self.adjlist = defaultdict(list) 
        self.nodes = nodes
        self.node_co = [[0, 0] for i in range(nodes)]
        
        ## search related data structures 
        self.visited = [False for i in range(nodes)]
        self.tsp_dist = [0.0 for i in range(nodes)]
        self.parent = [-1 for i in range(nodes)]
        self.best_path = []
        self.best_tsp_cost = 0.0
        self.largest_frontier = 0
        self.current_frontier = 0
        self.total_nodes_generated = 0
        
    def add_edge(self, src : int, dst : int, w : float = 1.0):
        # Assuming that the edge is bidirectional
        self.adjlist[src].append((dst, w))
        self.adjlist[dst].append((src, w))

    def display_adj_list(self) :
        for item in sorted(self.adjlist.items()):
            print("Node #", item[0], "Adjacents and Wieghts::")
            print(item[1])
        pass
    
    def check_if_collinear(self, x1 : int, y1 : int, current : int):
        for i in range(current):
            x2 = self.node_co[i][0]
            y2 = self.node_co[i][1]    
            for j in range(i + 1, current):
                x3 = self.node_co[j][0]
                y3 = self.node_co[j][1]
                area = x1 * (y2 - y3) + x2 * (y3 - y1) + x3 * (y1 - y2)
                # Make the visualization better...
                if abs(area) <= 2000:
                    return True
        return False

    def random_nodes(self):
        for i in range(self.nodes):
            (x, y) = (random.randint(1, Graph.MAX_XY_SCREEN), random.randint(1, Graph.MAX_XY_SCREEN))
            if i > 0:
                while True:
                    min_dist = Graph.distance(x, y, self.node_co[0][0], self.node_co[0][1])
                    for j in range(1, i):
                        new_dist = Graph.distance(x, y, self.node_co[j][0], self.node_co[j][1])
                        if new_dist < min_dist:
                            min_dist = new_dist
                    if min_dist < 100 or (i > 1 and self.check_if_collinear(x, y, i)):
                        (x, y) = (random.randint(1, Graph.MAX_XY_SCREEN), random.randint(1, Graph.MAX_XY_SCREEN))
                    else:
                        break
            
            self.node_co[i][0] = x
            self.node_co[i][1] = y
        pass

    def random_edges(self, edges):
        for i in range (edges):
            src = random.randint(0, self.nodes - 1)
            dest = src
            # Check if the edge is uniqe
            while src == dest or dest in self.adjlist[src]: 
                dest = random.randint(0, self.nodes - 1)
            weight = random.random() * Graph.MAX_EDGE_WEIGHT
            self.add_edge(src, dest, weight)
        pass

    @staticmethod
    def distance(x0 : int, y0 : int, x1 : int, y1 : int):
        return ((x0 - x1) ** 2 + (y0 - y1) ** 2) ** 0.5

    @staticmethod
    def create_random_graph(nodes = 10, edges = 20):
        g = Graph(nodes)
        g.random_nodes()
        g.random_edges(edges)
        g.display_adj_list()
        return g

    def _init_uninformed(self):
        self.best_path = []
        self.best_tsp_cost = float('inf')
        self.largest_frontier = 0
        self.current_frontier = 0  ### helpful only in dfs
        self.total_nodes_generated = 0
        for i in range(self.nodes):
            self.parent[i] = -1
            self.tsp_dist[i] = 0.0
            self.visited[i] = False
        
    def _generate_path(self, u):
        path = []
        while u != -1:
            path.append(u)
            u = self.parent[u]
        return path

    def _check_goal_node(self, u):
        ### GOAL NODE should have seen all nodes in it's path
        cnt_vis = sum([1 if value else 0 for value in self.visited])
        (v, w) = (-1, -1)
        ### AND ALSO it should have a connection with starting node i.e. node #0
        for _v, _w in self.adjlist[u]:
            if _v == 0:
                (v, w) = (_v, _w)
                break
        if cnt_vis == self.nodes and v != -1:
            # Save and update best path...
            cost = self.tsp_dist[u] + w
            if cost < self.best_tsp_cost:
                self.best_tsp_cost = cost
                self.best_path = self._generate_path(u)
            return True
        return False

    def _dfs(self, u):
        # Save statistics
        self.total_nodes_generated += 1
        self.current_frontier += 1
        self.largest_frontier = max(self.largest_frontier, self.current_frontier)
        self.visited[u] = True
        # Check goal node for TSP problem
        is_goal = self._check_goal_node(u)
        if not is_goal:
            # Continue search in this branch
            for (v, w) in self.adjlist[u]:
                if not self.visited[v]:
                    self.tsp_dist[v] = w + self.tsp_dist[u]
                    self.parent[v] = u
                    self._dfs(v)
                    
        self.visited[u] = False
        self.current_frontier = self.current_frontier - 1

    def dfs_tsp(self):
        self._init_uninformed()
        self._dfs(0)
        return self.best_path, self.best_tsp_cost, self.total_nodes_generated, self.largest_frontier

    def bfs_tsp(self):
        self._init_uninformed()
        frontier = []
        frontier.append((0, 0, [0]))
        self.total_nodes_generated = 1
        while len(frontier) > 0:
            current, cost, path = frontier.pop(0)
            #print(frontier)
            
            # Check for a goal node
            if len(path) == self.nodes:
                (v, w) = (-1, -1)
                ### Test connection with starting node i.e. node #0
                for _v, _w in self.adjlist[current]:
                    if _v == 0:
                        (v, w) = (_v, _w)
                        break
                cost += w
                if v != -1 and cost < self.best_tsp_cost:
                    self.best_path = path[::-1] # Reverse order of the path 
                    self.best_tsp_cost = cost
            else:
                # Expand and add nodes to the frontier
                for (v, w) in self.adjlist[current]:
                    if not v in path:
                        frontier.append((v, cost + w, path + [v]))
                        # Save statistics
                        self.total_nodes_generated += 1
                        self.largest_frontier = max(self.largest_frontier, len(frontier))

        return self.best_path, self.best_tsp_cost, self.total_nodes_generated, self.largest_frontier

    def _mst_prime_heuristic(self):
        # Constuct remaining cities
        remained_cities = [i for i in range(self.nodes) if not self.visited[i]]
        if len(remained_cities) == 0:
            return 0.0
        
        start = remained_cities[0]
        # Necessary data structures
        marked = {node:False for node in remained_cities}
        marked[start] = True
        mst_cost = 0

        # Save the open edges and their weights in a min heap
        open_edges = []
        # Add edges of the starting node the heap
        for (u, w) in self.adjlist[start]:
            if u in remained_cities:    # Only add edges that both cities are in remaining cities
                heappush(open_edges, (w, u))
        while open_edges:
            # Pop the smallest edge from the open edge to construct MST
            w, u = heappop(open_edges)
            # If it makes a loop in graph ignore this edge
            if marked[u]:
                continue
            # Add newly added node to the MST Tree
            marked[u] = True
            mst_cost += w
            # Add newly opend edges to the heap
            for (v, w) in self.adjlist[u]:
                if v in remained_cities:    # Only add edges that both cities are in remaining cities
                    heappush(open_edges, (w, v))

        total_cost = float('inf')
        # If the graph is connected
        if sum([1 if m[1] else 0 for m in marked.items()]) == len(remained_cities):
            total_cost = mst_cost
            # Find lightest edge from MST tree to node 0
            lightest = float('inf')
            for city in remained_cities:
                w = float('inf')
                for (_u, _w) in self.adjlist[city]:
                    if _u == 0:
                        w = _w
                        break
                lightest = min(lightest, w)
            # If the Tree isn't conntect the lightest and total_cost will be inf
            total_cost += lightest
        return total_cost

    def a_star_mst_tsp(self):
        self._init_uninformed()
        frontier = []
        heappush(frontier, (self._mst_prime_heuristic(), 0))
        self.total_nodes_generated = 1
        goal_found = False
        while frontier and not goal_found:
            print(frontier)
            estimated_cost, u = heappop(frontier)
            print(estimated_cost, u)
            if self.visited[u]:
                continue
            self.visited[u] = True
            
            # Check goal node
            #if self._check_goal_node(u):
            #    break
            #else:
            # Expand node
            for (v, w) in self.adjlist[u]:
                #if not self.visited[v] or self.tsp_dist[u] + w < self.tsp_dist[v]:
                if not self.visited[v] or self.tsp_dist[u] + w < self.tsp_dist[v]:
                    self.parent[v] = u
                    g = self.tsp_dist[v] = self.tsp_dist[u] + w
                    # Calcute the f estimate value for the newly expanded node
                    h = self._mst_prime_heuristic()
                    f = g + h
                    heappush(frontier, (f, v))
                    # Update the statistics
                    self.total_nodes_generated += 1
                    self.largest_frontier = max(self.largest_frontier, len(frontier))
                elif v == 0 and all(visit for visit in self.visited): ### Goal
                    goal_found = True
                    self.best_path = self._generate_path(u)
                    self.best_tsp_cost = self.tsp_dist[u] + w
                    print ("-----------------------------", self.visited)
                    break
    
        return self.best_path, self.best_tsp_cost, self.total_nodes_generated, self.largest_frontier
        

def show_path(root, path, graph, cost, largest, total, title=""):
    window = tk.Toplevel(root)
    window.title(title)
    c = tk.Canvas(window, height=600, width=600, bg='white')
    c.pack(fill=tk.BOTH, expand=True)
    # Get current width and height of canvas
    w = c.winfo_reqwidth()
    h = c.winfo_reqheight()
    print (w, " ", h)
    c.delete('grid_line') # Will only remove the grid_line
    # Print simple graph
    for i in range(graph.nodes):
        x, y = graph.node_co[i]
        c.create_oval(x, y, x + 10, 10 + y, fill='black')
        for edge in graph.adjlist[i]:
            v, w = edge
            c.create_line(graph.node_co[i][0] + 5, 5 + graph.node_co[i][1], graph.node_co[v][0] + 5, 5 + graph.node_co[v][1], fill='red', width = 4)
    
    # Make Start node different
    c.create_oval(graph.node_co[0][0], graph.node_co[0][1], graph.node_co[0][0] + 15, 15 + graph.node_co[0][1], fill='gray')
    
    # Show path edges with blue dashed color
     # The path is in the reverse order
    prev = 0
    print(path)
    for v in path:
        c.create_line(graph.node_co[prev][0] + 5, 5 + graph.node_co[prev][1], graph.node_co[v][0] + 5, 5 + graph.node_co[v][1], 
            fill='blue', width = 1, activedash=True, activewidth=3)
        prev = v

    l1 = ttk.Label(window, text="Cost of TSP: " + str(cost))
    l2 = ttk.Label(window, text="Largest size of frontier: " + str(largest))
    l3 = ttk.Label(window, text="Total number of nodes generated: " + str(total))
    l1.pack(ipadx=10, ipady=10)
    l2.pack(ipadx=10, ipady=10)
    l3.pack(ipadx=10, ipady=10)
    pass


def random_color():
    return random.randint(0,0x1000000)


def show_graph(graph, title=""):
    root = tk.Tk()
    root.title(title)
    c = tk.Canvas(root, height=600, width=600, bg='white')
    c.pack(fill=tk.BOTH, expand=True)

    w = c.winfo_reqwidth() # Get current width of canvas
    h = c.winfo_reqheight() # Get current height of canvas
    c.delete('grid_line') # Will only remove the grid_line

    for i in range(graph.nodes):
        c.create_oval(graph.node_co[i][0], graph.node_co[i][1], graph.node_co[i][0] + 10, 10 + graph.node_co[i][1], fill='black')
        color = '{:06x}'.format(random_color())
        for edge in graph.adjlist[i]:
            v = edge[0]
            # All edges of each nodes have the same color
            c.create_line(graph.node_co[i][0] + 5, 5 + graph.node_co[i][1], graph.node_co[v][0] + 5, 5 + graph.node_co[v][1], fill = ('#' + color), width = 3)
    return root


### 1.1
g = Graph.create_random_graph(nodes=random.randint(6, 10), edges=20)
root = show_graph(g, title="1.1 Original Random Graph")

### 1.2, 1.4 DFS
path, cost, total, largest = g.dfs_tsp()
print("1.2, 1.4 DFS TSP COST = ", cost)
message = "TSP with DFS Result"
if len(path) == 0:
    message += " Doesn't Exist..." # Some nodes with the Odd degrees
show_path(root, path, g, cost, largest, total, title=message)

### 1.2, 1.4 BFS
path, cost, total, largest = g.bfs_tsp()
print("1.2, 1.4 BFS TSP COST = ", cost)
message = "TSP with BFS Result"
if len(path) == 0:
    message += " Doesn't Exist..." # Some nodes with the Odd degrees
show_path(root, path, g, cost, largest, total, title=message)

### 1.5 A* + MST Heuristic
path, cost, total, largest = g.a_star_mst_tsp()
print("1.5 A* + MST Heuristic = ", cost)
message = "A* + MST Heuristic Result"
if len(path) == 0:
    message += " Doesn't Exist..." # Some nodes with the Odd degrees
show_path(root, path, g, cost, largest, total, title=message)

root.mainloop()

Node # 0 Adjacents and Wieghts::
[(2, 55.66996337785379), (8, 84.29673591717312), (7, 32.66738982593655), (5, 95.21534554600525), (8, 81.77799193104958), (8, 21.062110228052155)]
Node # 1 Adjacents and Wieghts::
[(6, 22.262064775555825), (4, 40.434506108131174), (5, 84.7226808316068), (5, 26.289190339574496)]
Node # 2 Adjacents and Wieghts::
[(7, 22.68857649804493), (4, 35.38784408025491), (0, 55.66996337785379), (4, 82.92584901695675)]
Node # 3 Adjacents and Wieghts::
[(5, 8.022838483318074)]
Node # 4 Adjacents and Wieghts::
[(1, 40.434506108131174), (2, 35.38784408025491), (8, 32.609425186031594), (6, 41.59845343424319), (5, 62.127676682725664), (2, 82.92584901695675)]
Node # 5 Adjacents and Wieghts::
[(8, 25.590881017061285), (1, 84.7226808316068), (3, 8.022838483318074), (1, 26.289190339574496), (4, 62.127676682725664), (0, 95.21534554600525)]
Node # 6 Adjacents and Wieghts::
[(1, 22.262064775555825), (4, 41.59845343424319), (8, 83.31695196870233)]
Node # 7 Adjacents and Wieghts::
